In [18]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
import matplotlib.pyplot as plt
from array import array
import os
from PIL import Image, ImageDraw
import sys
import time
import os

'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = os.environ["VISION_KEY"]
endpoint = os.environ["VISION_ENDPOINT"]
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

'\nEND - Authenticate\n'

In [19]:
 # Authenticate Azure AI Vision client
 cv_client = ImageAnalysisClient(
     endpoint=endpoint,
     credential=AzureKeyCredential(subscription_key)
 )

with open("D:/FACULTATE/SEM4/AI/Lab3/test2.jpeg", "rb") as image_file:
    image_data = image_file.read()

result = cv_client.analyze(
     image_data=image_data,
     visual_features=[VisualFeatures.READ]
 )

resultRead = ""
if result.read is not None:
     print("\nText:")

     # Prepare image for drawing
     image = Image.open("D:/FACULTATE/SEM4/AI/Lab3/test2.jpeg")

     for line in result.read.blocks[0].lines:
         # Return the text detected in the image
         print(f"  {line.text}")    
         resultRead += str(line.text)+" "
 
         for word in line.words:
             print(f"    {word.text}")
          
     print(resultRead)


Text:
  Succes in rezolvarea
    Succes
    in
    rezolvarea
  TEMELOR la
    TEMELOR
    la
  LABORA toarele de
    LABORA
    toarele
    de
  Inteligentà Artificialà!
    Inteligentà
    Artificialà!
Succes in rezolvarea TEMELOR la LABORA toarele de Inteligentà Artificialà! 


In [54]:
#1. calitatea procesului de recunoastere a textului, atat la nivel de caracter, cat si la nivel de cuvant
#a. prin folosirea unei metrici de distanta sau
#b. prin folosirea mai multor metrici de distanta.

In [20]:
print(resultRead)
result_cuvinte_FP = resultRead.split()
result_cuvinte = resultRead.split()
del result_cuvinte[5]
del result_cuvinte[5]
result_cuvinte.insert(5,"LABORA toarele")
print(result_cuvinte)
result_cuvinteCP = " ".join(result_cuvinte)
print(result_cuvinteCP)

Succes in rezolvarea TEMELOR la LABORA toarele de Inteligentà Artificialà! 
['Succes', 'in', 'rezolvarea', 'TEMELOR', 'la', 'LABORA toarele', 'de', 'Inteligentà', 'Artificialà!']
Succes in rezolvarea TEMELOR la LABORA toarele de Inteligentà Artificialà!


In [21]:
import Levenshtein
resultat_asteptat = "Succes în rezolvarea tEMELOR la LABORAtoarele de Inteligență Artificială!"
resultat_asteptat_cuvinte = resultat_asteptat.split()

cuvant_distantaFP = [Levenshtein.distance(cuvant_asteptat, cuvant_de_recunoscut) for cuvant_asteptat, cuvant_de_recunoscut in zip(resultat_asteptat_cuvinte, result_cuvinte_FP)]
cuvant_distanta = [Levenshtein.distance(cuvant_asteptat, cuvant_de_recunoscut) for cuvant_asteptat, cuvant_de_recunoscut in zip(resultat_asteptat_cuvinte, result_cuvinte)]  
nivel_caracter = Levenshtein.distance(resultat_asteptat, resultRead)


print("a. prin folosirea unei metrici de distanta")
print()
print("Inainte de prelucrarea output:")
print("Prin Levenshtein cuvinte",cuvant_distantaFP)
print("Prin Levenshtein caracter",nivel_caracter)
print("Dupa prelucrarea output:")
print("Prin Levenshtein cuvinte",cuvant_distanta)
print("Prin Levenshtein caracter",nivel_caracter)

a. prin folosirea unei metrici de distanta

Inainte de prelucrarea output:
Prin Levenshtein cuvinte [0, 1, 0, 1, 0, 7, 6, 10, 10]
Prin Levenshtein caracter 7
Dupa prelucrarea output:
Prin Levenshtein cuvinte [0, 1, 0, 1, 0, 1, 0, 2, 1]
Prin Levenshtein caracter 7


In [22]:
print("b. prin folosirea mai multor metrici de distanta.")
print()
distanta_cuvinte = 0
for i in range(len(resultat_asteptat)):
        # Incrementăm distanța dacă caracterele de pe aceeași poziție sunt diferite
    if result_cuvinteCP[i] != resultat_asteptat[i]:
            distanta_cuvinte += 1

distanta_caracter = 0
for linie_gen, linie_asteptata in zip(result_cuvinteCP, resultat_asteptat):
    for car_gen, car_asteptat in zip(linie_gen, linie_asteptata):
        distanta_caracter += car_gen != car_asteptat
        
print("Rezultat asteptat")
print(resultat_asteptat)
print("Rezultat generat")
print(result_cuvinteCP)
print("Prin Hamming cuvinte",distanta_cuvinte)
print("Prin Hamming caracter",distanta_caracter)

b. prin folosirea mai multor metrici de distanta.

Rezultat asteptat
Succes în rezolvarea tEMELOR la LABORAtoarele de Inteligență Artificială!
Rezultat generat
Succes in rezolvarea TEMELOR la LABORA toarele de Inteligentà Artificialà!
Prin Hamming cuvinte 37
Prin Hamming caracter 37


In [27]:
import jellyfish
distanta_jaro_winkler = jellyfish.jaro_winkler_similarity(resultat_asteptat, resultRead)

print("Rezultat asteptat")
print(resultat_asteptat)
print("Rezultat generat")
print(resultRead)
print("Similaritate Jaro-Winkler ",distanta_jaro_winkler)

Rezultat asteptat
Succes în rezolvarea tEMELOR la LABORAtoarele de Inteligență Artificială!
Rezultat generat
Succes in rezolvarea TEMELOR la LABORA toarele de Inteligentà Artificialà! 
Similaritate Jaro-Winkler  0.9469541393686718


In [28]:
n = len(resultRead)
m = len(resultat_asteptat)
scores = [[0] * m for _ in range(n)]
for i in range(n):
    for j in range(m):
        str1 = resultRead[i]
        str2 = resultat_asteptat[j]
        if len(str1) != len(str2):
            scores[i][j] = float('inf')  # nu se poate calcula distanța Levenshtein
        else:
            score = sum(Levenshtein.distance(c1, c2) for c1, c2 in zip(str1, str2))
            scores[i][j] = score
print("Rezultat asteptat")
print(resultat_asteptat)
print("Rezultat generat")
print(resultRead)
print("Nivel Longest common subsequence",score)

Rezultat asteptat
Succes în rezolvarea tEMELOR la LABORAtoarele de Inteligență Artificială!
Rezultat generat
Succes in rezolvarea TEMELOR la LABORA toarele de Inteligentà Artificialà! 
Nivel Longest common subsequence 1
